In [1]:
import os 
import json
import csv

Create directory to save human evaluation sheets

In [2]:
if not os.path.exists('human'):
    os.mkdir('human')

# Botsdobem

Count test entries per domain

In [3]:
from collections import Counter

path = os.path.join('results', 'botsdobem_mt5', 'result.json')
data = json.load(open(path))
domains = [w['intent'].split()[0] for w in data]
Counter(domains)

Counter({'[AMAZON_FIRE]': 54,
         '[COVID19]': 54,
         '[DEFORESTATION_DAILY]': 237,
         '[DEFORESTATION_MONTH]': 11})

Choose entries for human evaluation

30 for Amazon Fire, 30 for COVID19, 30 for deforestation daily and 10 for deforstation month

In [4]:
from random import shuffle

domains = {
    '[AMAZON_FIRE]': 40,
    '[COVID19]': 40,
    '[DEFORESTATION_DAILY]': 40,
    '[DEFORESTATION_MONTH]': 10
}
idxs = []

path = os.path.join('results', 'botsdobem_mt5', 'result.json')
data = json.load(open(path))
shuffle(data)

for row in data:
    idx = row['idx']
    intent = row['intent']
    domain = intent.split()[0]
    
    if domains[domain] > 0:
        idxs.append(idx)
        domains[domain] -= 1

fdir = os.path.join('human', 'botsdobem')
if not os.path.exists(fdir):
    os.mkdir(fdir)

# save idxs
path = os.path.join(fdir, 'idxs.json')
json.dump(idxs, open(path, 'w'), separators=(',', ':'), sort_keys=True, indent=4)

Get output of the model for the selected indexes

In [5]:
output = []
for fdir in os.listdir('results'):
    if 'botsdobem' in fdir:
        path = os.path.join('results', fdir, 'result.json')
        result = json.load(open(path))
        
        for idx in idxs:
            f = [w for w in result if w['idx'] == idx][0]
            output.append({
                'idx': idx,
                'intent': f['intent'],
                'hyp': f['hyp'].strip(),
                'ref': f['refs'][0],
                'domain': fdir
            })
            
for idx in idxs:
    f = [w for w in result if w['idx'] == idx][0]
    output.append({
        'idx': idx,
        'intent': f['intent'],
        'hyp': f['refs'][0],
        'ref': f['refs'][0],
        'domain': 'gold'
    })

shuffle(output)
# save idxs
fdir = os.path.join('human', 'botsdobem')
path = os.path.join(fdir, 'result.json')
json.dump(output, open(path, 'w'), separators=(',', ':'), sort_keys=True, indent=4)

Save results in a csv

In [6]:
fdir = os.path.join('human', 'botsdobem')
path = os.path.join(fdir, 'annotation.csv')

header = ["MR", "Reference", "Hypothesis", \
          "Incorrect Number", "Incorrect Named Entity", \
          "Incorrect Word", "Context Error", "Not checkable", "Other", "Fluency"]

with open(path, 'w') as f:
    writer = csv.writer(f, delimiter=';', quotechar='\"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(header)
    
    for row in output:
        writer.writerow([row['intent'], row['ref'], row['hyp']])